# Toy example to demonstrate the use of **mescal**

In [1]:
import os
import pandas as pd
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing
from mescal import *
import bw2data as bd

In [2]:
ei_version = '3.10.1'

In [3]:
# AMPL licence
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL license file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [4]:
path_model = './data/esm/' # Path to the energy system model
path_model_lca = './data/esm/lca/'
path_inputs = './data/lca/' # Path to the LCA data
path_results = './results/' # Path to the results

## Generating LCA impact scores

In [5]:
mapping = pd.read_csv(path_inputs+'mapping.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')
model = pd.read_csv(path_inputs+'model.csv')

In [6]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version}')

In [7]:
name_main_database = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_2050'

In [8]:
main_db = Database(name_main_database, create_pickle=True)

2025-05-08 17:20:49,782 - Database - INFO - Loaded ecoinvent_cutoff_3.10.1_image_SSP2-Base_2050 from pickle!


In [9]:
# Add CPC categories to the main database
main_db.add_CPC_categories()

In [10]:
ranking_best_ecoinvent_locations = ['GLO', 'RoW']

In [11]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    main_database=main_db,
    esm_db_name='Tatooine_2050',

    # Optional inputs
    technology_compositions=techno_compositions,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=False,
    locations_ranking=ranking_best_ecoinvent_locations,
    esm_location='GLO',
    results_path_file=path_results,
)

In [12]:
esm.check_inputs()

2025-05-08 17:20:51,914 - Mescal - WARNING - List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value: ['RES_SOLAR', 'RES_WIND_ONSHORE']
2025-05-08 17:20:51,916 - Mescal - WARNING - List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow): ['BATTERY', 'GRID']


In [13]:
# Adapt mapping file to ESM location
esm.change_location_mapping_file()
# esm.mapping.to_csv(path_inputs+f'mapping.csv', index=False)

2025-05-08 17:20:52,168 - Mescal - WARNING - No location found in your ranking for electricity, high voltage - electricity production, at natural gas-fired combined cycle power plant, post, pipeline 200km, storage 1000m. Have to keep the initial location: RER
2025-05-08 17:20:52,184 - Mescal - WARNING - No location found in your ranking for electricity, high voltage - electricity production, at hard coal-fired IGCC power plant. Have to keep the initial location: RER
2025-05-08 17:20:52,300 - Mescal - WARNING - No location found in your ranking for electricity, high voltage - market for electricity, high voltage. Have to keep the initial location: SA


In [14]:
missing_flows = main_db.test_mapping_file(esm.mapping)

2025-05-08 17:20:52,362 - Database - INFO - Mapping successfully linked to the database


In [15]:
esm.create_esm_database()

2025-05-08 17:20:53,257 - Mescal - INFO - Starting to add construction and resource activities database
2025-05-08 17:20:53,327 - Mescal - INFO - Construction and resource activities added to the database in 0.1 seconds
2025-05-08 17:20:53,327 - Mescal - INFO - Starting to remove double-counted flows
100%|██████████| 7/7 [00:00<00:00, 216.43it/s]
2025-05-08 17:20:53,478 - Mescal - INFO - Double-counting removal done in 0.2 seconds
2025-05-08 17:20:53,561 - Mescal - INFO - Starting to correct efficiency differences
2025-05-08 17:20:53,967 - Mescal - INFO - Efficiency differences corrected in 0.4 seconds
2025-05-08 17:20:53,981 - Mescal - INFO - Starting to write database
2025-05-08 17:20:54,221 - Database - INFO - Previous Tatooine_2050 will be overwritten!
Writing activities to SQLite3 database:
0% [######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/08/2025 17:20:55
  Finished: 05/08/2025 17:20:55
  Total time elapsed: 00:00:00
  CPU %: 89.60
  Memory %: 12.14


2025-05-08 17:20:56,164 - Database - INFO - Tatooine_2050 written to Brightway!
2025-05-08 17:20:56,173 - Mescal - INFO - Database written in 2.2 seconds


In [16]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(path_results+'mapping_with_new_codes.csv', index=False)

In [17]:
methods = ['IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10']

In [18]:
R_long = esm.compute_impact_scores(
    methods=methods,
    impact_abbrev=impact_abbrev,
)

Getting activity data


100%|██████████| 22/22 [00:00<00:00, 10920.08it/s]


Adding exchange data to activities


100%|██████████| 753/753 [00:00<00:00, 35956.50it/s]


Filling out exchange data


100%|██████████| 22/22 [00:00<00:00, 87.04it/s]
2025-05-08 17:20:56,669 - Database - INFO - Loaded Tatooine_2050 from brightway!
20it [00:03,  5.38it/s]


In [19]:
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [20]:
activities_subject_to_double_counting = pd.read_csv(f'{path_results}activities_subject_to_double_counting.csv')

In [21]:
R_long_direct_emissions = esm.compute_impact_scores(
    methods=methods,
    assessment_type='direct emissions',
    impact_abbrev=impact_abbrev,
    activities_subject_to_double_counting=activities_subject_to_double_counting,
    overwrite=True,
)

Getting activity data


100%|██████████| 22/22 [00:00<00:00, 10945.99it/s]


Adding exchange data to activities


100%|██████████| 753/753 [00:00<00:00, 34328.35it/s]


Filling out exchange data


100%|██████████| 22/22 [00:00<00:00, 80.61it/s]
2025-05-08 17:28:50,364 - Database - INFO - Loaded Tatooine_2050 from brightway!
2025-05-08 17:28:50,446 - Database - INFO - Previous Tatooine_2050_direct_emissions will be overwritten!
Writing activities to SQLite3 database:
0% [#########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
2025-05-08 17:28:52,100 - Database - INFO - Tatooine_2050_direct_emissions written to Brightway!


Title: Writing activities to SQLite3 database:
  Started: 05/08/2025 17:28:51
  Finished: 05/08/2025 17:28:51
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 12.19


7it [00:00, 142.87it/s]


In [22]:
R_long_direct_emissions.to_csv(f'{path_results}impact_scores_direct_emissions.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [23]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': '2050',
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [24]:
specific_lcia_abbrev = ['m_CCS']

In [25]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=  path_model_lca,
    metadata=metadata,
    file_name='techs_lca',
)

In [26]:
# Create .dat file for direct emissions only
esm.normalize_lca_metrics(
    assessment_type='direct emissions',
    R=R_long_direct_emissions,
    max_per_cat=pd.read_csv(  path_model_lca + 'techs_lca_max.csv'),
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=  path_model_lca,
    metadata=metadata,
    file_name='techs_lca_direct',
)

In [27]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=  path_model_lca,
    metadata=metadata,
    file_name='objectives_lca',
)

In [28]:
# Create the .mod file for direct emissions only
esm.generate_mod_file_ampl(
    assessment_type='direct emissions',
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=  path_model_lca,
    metadata=metadata,
    file_name='objectives_lca_direct',
)

## Running the ESM

In [29]:
with open(path_model + 'objective_function.mod', 'w') as f:
        f.write('minimize obj: TotalLCIA_m_CCS;')

In [37]:
# Initialize the 2021 QC model with .mod and .dat files
model = Model(
    mod_files=[
        path_model+'main.mod',
        path_model_lca+'objectives_lca.mod',
        path_model_lca+'objectives_lca_direct.mod',
        path_model+'objective_function.mod',
    ],
    dat_files=[
        path_model+'data.dat',
        path_model+'techs.dat',
        path_model_lca+'techs_lca.dat',
        path_model_lca+'techs_lca_direct.dat',
    ],
)

In [38]:
# Define the solver options
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}

In [39]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [40]:
# Solve the model and get results
results = postprocessing(es.calc())

Gurobi 12.0.0: 

In [41]:
results.variables['Annual_Prod']

,Annual_Prod,Run
BATTERY,0.000620,0
CCGT,0.000000,0
CCGT_CC,0.000000,0
COAL_IGCC,0.000000,0
COAL_IGCC_CC,0.000000,0
GRID,0.000746,0
NUCLEAR,0.000000,0
PV,4000.000000,0
WIND_ONSHORE,0.000000,0


In [42]:
results.variables['F_Mult']

,F_Mult,Run
BATTERY,0.922760,0
CCGT,0.000000,0
CCGT_CC,0.000000,0
COAL_IGCC,0.000000,0
COAL_IGCC_CC,0.000000,0
GRID,1.036401,0
NUCLEAR,0.000000,0
PV,3.075866,0
WIND_ONSHORE,0.000000,0


In [36]:
# lyrio = results.parameters['layers_in_out'].reset_index()
# lyrio = lyrio[lyrio.layers_in_out != 0].drop(columns=['Run']).rename({'index0': 'Name', 'index1': 'Flow', 'layers_in_out': 'Amount'}, axis=1)
# lyrio.to_csv(path_inputs+'model.csv', index=False)

## Visualize the results